In [1]:
%display plain
#%display latex

In [2]:
from matplotlib import cm
import numpy as np
import itertools
from itertools import combinations
from __future__ import print_function

In [3]:
real_vars = var('r,s,t,theta,phi,x,y,z')
for rv in real_vars:
    assume(rv, 'real')

In [4]:
xs = var(['x{}'.format(n) for n in range(4)])
for xi in xs: assume(xi,'real')

ys = var(['y{}'.format(n) for n in range(3)])
for yi in ys: assume(yi,'real')

xs, ys

((x0, x1, x2, x3), (y0, y1, y2))

In [5]:
p1 = sum([xi   for xi in xs])
p2 = sum([xi^2 for xi in xs])
p3 = sum([xi^3 for xi in xs])
p4 = sum([xi^4 for xi in xs])
p5 = sum([xi^5 for xi in xs])

p1, p2, p3, p4, p5

(x0 + x1 + x2 + x3,
 x0^2 + x1^2 + x2^2 + x3^2,
 x0^3 + x1^3 + x2^3 + x3^3,
 x0^4 + x1^4 + x2^4 + x3^4,
 x0^5 + x1^5 + x2^5 + x3^5)

In [6]:
e1 = sum([ xi             for (xi,           ) in combinations(xs,1)])
e2 = sum([ xi*xj          for (xi,xj         ) in combinations(xs,2)])
e3 = sum([ xi*xj*xk       for (xi,xj,xk      ) in combinations(xs,3)])
e4 = sum([ xi*xj*xk*xl    for (xi,xj,xk,xl   ) in combinations(xs,4)])

e1, e2, e3, e4

(x0 + x1 + x2 + x3,
 x0*x1 + x0*x2 + x1*x2 + x0*x3 + x1*x3 + x2*x3,
 x0*x1*x2 + x0*x1*x3 + x0*x2*x3 + x1*x2*x3,
 x0*x1*x2*x3)

In [7]:
def inner_product(f,g,vrs):
    ip = (f.conjugate()*g).expand()*exp(-sum([vi^2 for vi in vrs]))/(2*pi)^len(vrs)
    # odd parts will integrate out, so remove them. makes this go quite a bit faster
    for vi in vrs:
        ip = ((ip+ip(vi=-vi))/2).simplify_full()
    for vi in vrs:
        ip = integrate(ip,vi,-oo,oo)
    return ip

In [8]:
com = vector([1,1,1,1])/sqrt(4) # center of mass

j1 = vector(QQ,[1,0,0,0])
j2 = vector(QQ,[0,1,0,0])
j3 = vector(QQ,[0,0,1,0])

j1 = j1 - (j1*com)*com/com.norm()^2
j2 = j2 - (j2*com)*com/com.norm()^2
j3 = j3 - (j3*com)*com/com.norm()^2

j2 = j2 - (j2*j1)*j1/j1.norm()^2
j3 = j3 - (j3*j1)*j1/j1.norm()^2

j3 = j3 - (j3*j2)*j2/j2.norm()^2

j1 = j1/j1.norm()
j2 = j2/j2.norm()
j3 = j3/j3.norm()

In [9]:
Pm = matrix([com,j1,j2,j3]).transpose()
Pl = matrix([j1,j2,j3]).transpose()

Qm = matrix([[0,0,0],
             [1,0,0],
             [0,1,0],
             [0,0,1]])

jacobi_projection = (Pl*Qm.transpose()*Pm^-1)

In [10]:
threebody_mats = [
    # 012
    matrix([[1, 1, 1, 1],
            [1,-1, 0, 0],
            [0, 1,-1, 0]]),
    # 013
    matrix([[1, 1, 1, 1],
            [1,-1, 0, 0],
            [0, 1, 0,-1]]),
    # 023
    matrix([[1, 1, 1, 1],
            [1, 0,-1, 0],
            [0, 0, 1,-1]]),
    # 123
    matrix([[1, 1, 1, 1],
            [0, 1,-1, 0],
            [0, 0, 1,-1]])
]


def orthogonalize_rows(M):
    rows = list(M)
    for idx in range(len(rows)):
        rows[idx] = (rows[idx]/rows[idx].norm()).simplify_full()
        for idx2 in range(idx+1,len(rows)):
            rows[idx2] = rows[idx2] - (rows[idx2]*rows[idx])*rows[idx]
    return matrix(rows)
    
# the (4 choose 3) 3-body interations in 4 dimensions
tbi4s = [ M.right_kernel().basis_matrix() for M in threebody_mats ]
tbi4s = [ orthogonalize_rows(M)[0] for M in tbi4s]

# the (4 choose 3) 3-body interations in 3 dimensions
tbi3s = [ (M*Pl).right_kernel().basis_matrix() for M in threebody_mats ]
tbi3s = [ orthogonalize_rows(M)[0] for M in tbi3s]

In [11]:
f1 = e1(*(Pm*Qm*vector(ys))).simplify_full().expand()
f2 = e2(*(Pm*Qm*vector(ys))).simplify_full().expand()
f3 = e3(*(Pm*Qm*vector(ys))).simplify_full().expand()
f4 = e4(*(Pm*Qm*vector(ys))).simplify_full().expand()

f1, f2, f3, f4

(0,
 -1/2*y0^2 - 1/2*y1^2 - 1/2*y2^2,
 1/18*sqrt(3)*sqrt(2)*y1^3 - 1/6*sqrt(3)*sqrt(2)*y1*y2^2 + 1/9*sqrt(3)*y0^3 - 1/6*sqrt(3)*y0*y1^2 - 1/6*sqrt(3)*y0*y2^2,
 1/12*sqrt(2)*y0*y1^3 - 1/4*sqrt(2)*y0*y1*y2^2 - 1/48*y0^4 + 1/8*y0^2*y1^2 + 1/8*y0^2*y2^2)

In [12]:
def stereographic_proj(v):
    """ Standard Stereographic projection with possible pre-rotation about X, Y and Z axis"""
    w = vector(v)
    return (w[0]/(1-w[2]),w[1]/(1-w[2]))

def to_iso_spherical_coords(v):
    if type(v) == sage.matrix.matrix_symbolic_dense.Matrix_symbolic_dense:
        v = vector(v)
    x, y, z = list(v)
    r = sqrt(x^2+y^2+z^2)
    theta = arccos(z/r)
    phi = atan2(y,x)
    return (r,theta,phi)

def to_iso_sphere_coords(v):
    r, theta, phi = to_iso_spherical_coords(v)
    return (phi, theta) #(theta, phi)

In [13]:
def legendre_poly(n):
    return diff( (x^2-1)^n, x, n)/(2^n*factorial(n))

def assoc_legendre_poly(m,l):
    if m >= 0:
        return (-1)^m * (1-x^2)^(m/2) *diff(legendre_poly(l),x,m)
    else:
        return (-1)^m * factorial(l-m) /factorial(l+m) * assoc_legendre_poly(-m,l)

def real_sh(m,l):
    return r^(l)*cos(m*phi)*assoc_legendre_poly(m,l)(x=cos(theta))

def imag_sh(m,l):
    return r^(l)*sin(m*phi)*assoc_legendre_poly(m,l)(x=cos(theta))

def polar_to_euclid(f):
    return ( f(theta=arccos(y2/r)) )(r=sqrt(y0^2+y1^2+y2^2), phi=atan2(y1,y0))

def euclid_real(m,l):
    return polar_to_euclid(real_sh(m,l)).simplify()

def euclid_imag(m,l):
    return polar_to_euclid(imag_sh(m,l)).simplify()

def euclid_to_sphere(f):
    return f(y0=r*sin(theta)*cos(phi),y1=r*sin(theta)*sin(phi),y2=r*cos(theta))

def rotation_matrix(theta):
    return matrix([[cos(theta), -sin(theta)],
                   [sin(theta),  cos(theta)]])

In [14]:
def lap(f,vrs):
    """ laplacian """
    return sum([diff(f,vi,2) for vi in vrs])

def grad(f,vrs):
    return vector([diff(f,vr) for vr in vrs])

def rep_lap(f,k,vrs):
    """ repeated laplacian  """
    if k <= 0:
        return f
    elif k == 1:
        return lap(f,vrs)
    else:
        return lap(rep_lap(f,k-1,vrs),vrs)
    
def semi_factorial(n):
    """ product: n*(n-2)*(n-4)*...*1 """
    if n <= 1:
        return 1
    else:
        return n*semi_factorial(n-2)
    
def harmonic_leading_term(f,n,vrs):
    """ as in (1-33), pg 8 of Avery's Hyperspherical Harmonics, Applications in Quantum Theory """
    r_sq = sum([vi^2 for vi in vrs])
    d = len(vrs)
    return sum([ ((-1)^k)*semi_factorial(d+2*n-2*k-4) \
                / ( semi_factorial(2*k) * semi_factorial(d+2*n-4) ) \
                * (r_sq^k) * rep_lap(f,k,vrs) for k in range(floor(n/2)+1)])

In [15]:
# a harmonic symmetric function of degree 1 in 4 variables
sym_fn_1 = 0
ell = 1

for vec in [vector([1,1,1,1])]: #jacobi_projection:
    sym_fn_1 += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()

# check symmetry
assert (sym_fn_1(x0=x1,x1=x0)-sym_fn_1).expand() == 0
assert (sym_fn_1(x0=x2,x2=x0)-sym_fn_1).expand() == 0
assert (sym_fn_1(x0=x3,x3=x0)-sym_fn_1).expand() == 0

# is harmonic
assert lap(sym_fn_1,xs).simplify_full() == 0

sym_fn_1.expand()

x0 + x1 + x2 + x3

In [16]:
# a harmonic symmetric function of degree 2 in 4 variables
sym_fn_2 = 0
ell = 2
# when we use (1,1,1,1) for the vector, we get 2*e_2
# when we use the jacobi coords, -8/3*e_2
for vec in jacobi_projection:
    sym_fn_2 += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()

# check symmetry
assert (sym_fn_2(x0=x1,x1=x0)-sym_fn_2).expand() == 0
assert (sym_fn_2(x0=x2,x2=x0)-sym_fn_2).expand() == 0
assert (sym_fn_2(x0=x3,x3=x0)-sym_fn_2).expand() == 0

# is harmonic
assert lap(sym_fn_2,xs).simplify_full() == 0

sym_fn_2.expand()

-8/3*x0*x1 - 8/3*x0*x2 - 8/3*x1*x2 - 8/3*x0*x3 - 8/3*x1*x3 - 8/3*x2*x3

In [17]:
(sym_fn_2+sym_fn_1^2).expand()

x0^2 - 2/3*x0*x1 + x1^2 - 2/3*x0*x2 - 2/3*x1*x2 + x2^2 - 2/3*x0*x3 - 2/3*x1*x3 - 2/3*x2*x3 + x3^2

In [18]:
# a harmonic symmetric function of degree 3 in 4 variables
sym_fn_3A = 0
ell = 3
for vec in list(identity_matrix(4)):
    sym_fn_3A += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()

# check symmetry
assert (sym_fn_3A(x0=x1,x1=x0)-sym_fn_3A).expand() == 0
assert (sym_fn_3A(x0=x2,x2=x0)-sym_fn_3A).expand() == 0
assert (sym_fn_3A(x0=x3,x3=x0)-sym_fn_3A).expand() == 0

# is harmonic
assert lap(sym_fn_3A,xs).simplify_full() == 0

sym_fn_3A.expand()

4*x0^3 - 4*x0^2*x1 - 4*x0*x1^2 + 4*x1^3 - 4*x0^2*x2 - 4*x1^2*x2 - 4*x0*x2^2 - 4*x1*x2^2 + 4*x2^3 - 4*x0^2*x3 - 4*x1^2*x3 - 4*x2^2*x3 - 4*x0*x3^2 - 4*x1*x3^2 - 4*x2*x3^2 + 4*x3^3

In [19]:
# a harmonic symmetric function of degree 3 in 4 variables
sym_fn_3B = 0
ell = 3
for vec in jacobi_projection:
    sym_fn_3B += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()

# check symmetry
assert (sym_fn_3B(x0=x1,x1=x0)-sym_fn_3B).expand() == 0
assert (sym_fn_3B(x0=x2,x2=x0)-sym_fn_3B).expand() == 0
assert (sym_fn_3B(x0=x3,x3=x0)-sym_fn_3B).expand() == 0

# is harmonic
assert lap(sym_fn_3B,xs).simplify_full() == 0

sym_fn_3B.expand()

8/3*sqrt(3)*x0^3 - 8/3*sqrt(3)*x0^2*x1 - 8/3*sqrt(3)*x0*x1^2 + 8/3*sqrt(3)*x1^3 - 8/3*sqrt(3)*x0^2*x2 + 16/3*sqrt(3)*x0*x1*x2 - 8/3*sqrt(3)*x1^2*x2 - 8/3*sqrt(3)*x0*x2^2 - 8/3*sqrt(3)*x1*x2^2 + 8/3*sqrt(3)*x2^3 - 8/3*sqrt(3)*x0^2*x3 + 16/3*sqrt(3)*x0*x1*x3 - 8/3*sqrt(3)*x1^2*x3 + 16/3*sqrt(3)*x0*x2*x3 + 16/3*sqrt(3)*x1*x2*x3 - 8/3*sqrt(3)*x2^2*x3 - 8/3*sqrt(3)*x0*x3^2 - 8/3*sqrt(3)*x1*x3^2 - 8/3*sqrt(3)*x2*x3^2 + 8/3*sqrt(3)*x3^3

In [20]:
# a harmonic symmetric function of degree 3 in 4 variables
sym_fn_3C = 0
ell = 3
for vec in [vector([1,1,1,1])]:
    sym_fn_3C += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()

# check symmetry
assert (sym_fn_3C(x0=x1,x1=x0)-sym_fn_3C).expand() == 0
assert (sym_fn_3C(x0=x2,x2=x0)-sym_fn_3C).expand() == 0
assert (sym_fn_3C(x0=x3,x3=x0)-sym_fn_3C).expand() == 0

# is harmonic
assert lap(sym_fn_3C,xs).simplify_full() == 0

sym_fn_3C.expand()

-x0^3 + x0^2*x1 + x0*x1^2 - x1^3 + x0^2*x2 + 6*x0*x1*x2 + x1^2*x2 + x0*x2^2 + x1*x2^2 - x2^3 + x0^2*x3 + 6*x0*x1*x3 + x1^2*x3 + 6*x0*x2*x3 + 6*x1*x2*x3 + x2^2*x3 + x0*x3^2 + x1*x3^2 + x2*x3^2 - x3^3

In [21]:
# looks like the jacobi coordinates are the way to go
sym_fn_3A(*(t*com)), sym_fn_3B(*(t*com)), sym_fn_3C(*(t*com))

(-4*t^3, 0, 4*t^3)

In [22]:
# All differ by a multiple of e_3
(sym_fn_3A/4-sym_fn_3B/(8/sqrt(3))).expand(), (sym_fn_3A/4-sym_fn_3C/(-1)).expand(), (sym_fn_3B/(8/sqrt(3))-sym_fn_3C/(-1)).expand()

(-2*x0*x1*x2 - 2*x0*x1*x3 - 2*x0*x2*x3 - 2*x1*x2*x3,
 6*x0*x1*x2 + 6*x0*x1*x3 + 6*x0*x2*x3 + 6*x1*x2*x3,
 8*x0*x1*x2 + 8*x0*x1*x3 + 8*x0*x2*x3 + 8*x1*x2*x3)

In [23]:
# killing the center of mass term in A/C gives a multiple of B
((sym_fn_3A + sym_fn_3C)/3).expand() - (sym_fn_3B/(8/sqrt(3))).expand()

0

In [24]:
# a harmonic symmetric function of degree 4 in 4 variables
sym_fn_4 = 0
ell = 4
for vec in jacobi_projection:
    sym_fn_4 += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()


sym_fn_4 -= (r^ell*gegenbauer(ell,4/2-1,
                              (vector([1,1,1,1])/2)*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()/5*4

sym_fn_4 *= 15/32
    
# check symmetry
assert (sym_fn_4(x0=x1,x1=x0)-sym_fn_4).expand() == 0
assert (sym_fn_4(x0=x2,x2=x0)-sym_fn_4).expand() == 0
assert (sym_fn_4(x0=x3,x3=x0)-sym_fn_4).expand() == 0

# is harmonic
assert lap(sym_fn_4,xs).simplify_full() == 0

sym_fn_4 = sym_fn_4.expand()

sym_fn_4

x0^4 - 4/3*x0^3*x1 - 2*x0^2*x1^2 - 4/3*x0*x1^3 + x1^4 - 4/3*x0^3*x2 + 4*x0^2*x1*x2 + 4*x0*x1^2*x2 - 4/3*x1^3*x2 - 2*x0^2*x2^2 + 4*x0*x1*x2^2 - 2*x1^2*x2^2 - 4/3*x0*x2^3 - 4/3*x1*x2^3 + x2^4 - 4/3*x0^3*x3 + 4*x0^2*x1*x3 + 4*x0*x1^2*x3 - 4/3*x1^3*x3 + 4*x0^2*x2*x3 - 24*x0*x1*x2*x3 + 4*x1^2*x2*x3 + 4*x0*x2^2*x3 + 4*x1*x2^2*x3 - 4/3*x2^3*x3 - 2*x0^2*x3^2 + 4*x0*x1*x3^2 - 2*x1^2*x3^2 + 4*x0*x2*x3^2 + 4*x1*x2*x3^2 - 2*x2^2*x3^2 - 4/3*x0*x3^3 - 4/3*x1*x3^3 - 4/3*x2*x3^3 + x3^4

In [25]:
sym_fn_4(*(t*com))

0

In [70]:
def num_partitions(n,coins):
    counts = [0]*(n+1)
    vec_coins = vector(coins)
    for v in itertools.product(range(n+1),repeat=len(coins)):
        value = vector(v) * vec_coins
        if value <= n:
            counts[value] += 1
    return counts

partitions = num_partitions(12,coins=[1,2,3,4])

harmonic_count = [ partitions[n] - partitions[n-2] if n>2 else partitions[n] for n in range(len(partitions))]

for n, count in enumerate(harmonic_count):
    print(n, '\t', count, '\t', partitions[n])

0 	 1 	 1
1 	 1 	 1
2 	 2 	 2
3 	 2 	 3
4 	 3 	 5
5 	 3 	 6
6 	 4 	 9
7 	 5 	 11
8 	 6 	 15
9 	 7 	 18
10 	 8 	 23
11 	 9 	 27
12 	 11 	 34


In [26]:
# a harmonic symmetric function of degree 12 in 4 variables
sym_fn_12 = 0
ell = 12
for vec in jacobi_projection:
    sym_fn_12 += (r^ell*gegenbauer(ell,4/2-1,
                                  (vec/norm(vec))*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()

sym_fn_12 -= (r^ell*gegenbauer(ell,4/2-1,
                               (vector([1,1,1,1])/2)*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()/13*4
    
# check symmetry
assert (sym_fn_12(x0=x1,x1=x0)-sym_fn_12).expand() == 0
assert (sym_fn_12(x0=x2,x2=x0)-sym_fn_12).expand() == 0
assert (sym_fn_12(x0=x3,x3=x0)-sym_fn_12).expand() == 0

# is harmonic
assert lap(sym_fn_12,xs).simplify_full() == 0

In [27]:
sym_fn_12(*(t*com))

0

4096/3465*t^12

In [49]:
sym_com_12 = (r^ell*gegenbauer(ell,4/2-1,
                               (vector([1,1,1,1])/2)*(vector(xs)/norm(vector(xs)))))\
    (r=sqrt(sum([xi^2 for xi in xs]))).simplify_full()/13*(4096/3465)

sym_com_12 /= sym_com_12(*com)

In [51]:
harm3_4 = harmonic_leading_term( sym_fn_3B^4, 12, xs ) #harmonic_leading_term( e3^4, 12, xs )

harm3_4 -= sym_com_12*harm3_4(*com)

harm3_4(*(t*com))

0

In [52]:
harm4_3 = harmonic_leading_term( sym_fn_4^3, 12, xs ) #harmonic_leading_term( e4^3, 12, xs )

harm4_3 -= sym_com_12*harm4_3(*com)

harm4_3(*(t*com))

0

In [62]:
# degree 12 harmonic functions
#harm1_12 = harmonic_leading_term( e1^12, 12, xs )
harm1_12 = sym_fn_12
harm1_12(*(t*com))

0

In [59]:
coeffs = var(["c{}".format(n) for n in range(3)])

print(matrix([[harm1_12(*com), harm1_12(*tbi4s[0])],
              [ harm3_4(*com),  harm3_4(*tbi4s[0])],
              [ harm4_3(*com),  harm4_3(*tbi4s[0])]]))

solns = solve(list(vector(coeffs)*matrix([[harm1_12(*com), harm1_12(*tbi4s[0])],
                                         [ harm3_4(*com),  harm3_4(*tbi4s[0])],
                                         [ harm4_3(*com),  harm4_3(*tbi4s[0])]])),coeffs)[0]

print(solns)

free_vr = list(set().union(*[ soln.rhs().variables() for soln in solns ]))[0]

coeffs = [soln.rhs()({free_vr: 1}) for soln in solns]

wv4d = (harm1_12*coeffs[0] + harm3_4*coeffs[1] + harm4_3*coeffs[2]).expand().simplify_full()

#wv4d

[                  0    27102208/2302911]
[                  0 4447535104/32837805]
[                  0       106528/729729]
[c0 == -808947/65214688*r3 - 117268992/10189795*r4, c1 == r4, c2 == r3]


In [60]:
wv4d(*(t*com + s*tbi4s[0])).expand()

1849473413948/601697204955*sqrt(3)*s^13 + 572497203736/5730449571*sqrt(3)*s^11*t^2 - 1281169462508/2025916515*sqrt(3)*s^9*t^4 + 9217074628624/7428360555*sqrt(3)*s^7*t^6 - 6824784792412/7428360555*sqrt(3)*s^5*t^8 + 13838495320/70746291*sqrt(3)*s^3*t^10 - 9544884/10189795*sqrt(3)*s*t^12 + 8630392180/13371048999*s^12*t + 121971863000/1910149857*s^10*t^3 - 5396901186844/22285081665*s^8*t^5 + 129721447888/495224037*s^6*t^7 + 2823270284/353731455*s^4*t^9 - 16371882344/275124465*s^2*t^11 + 9544884/10189795*t^13 - 30640810075957/61613793787392*sqrt(3)*s^11*t + 84028803914807/20537931262464*sqrt(3)*s^9*t^3 - 6494428296997/3422988543744*sqrt(3)*s^7*t^5 - 1671742189165/1140996181248*sqrt(3)*s^5*t^7 + 1827197545/3292918272*sqrt(3)*s^3*t^9 - 4235311/1043435008*sqrt(3)*s*t^11 - 7799381099225/22405015922688*s^12 + 4239905452471/2161887501312*s^10*t^2 - 55410440090351/9127969449984*s^8*t^4 + 13151772741827/2281992362496*s^6*t^6 - 9195692677/6899447808*s^4*t^8 - 127145123/1149907968*s^2*t^10 + 12705933

In [ ]:
# symmetry
assert wv4d(x0=x1,x1=x0)-wv4d == 0
assert wv4d(x0=x2,x2=x0)-wv4d == 0
assert wv4d(x0=x3,x3=x0)-wv4d == 0

In [ ]:
# harmonic
lap(wv4d,xs).expand()

In [ ]:
# vanishes on three body interations
print(wv4d(*com))

for v in tbi4s:
    print(wv4d(*(t*v)))

In [ ]:
wv3d = (wv4d(*(Pm*Qm*vector(ys)))).simplify_full().expand()

wv3d

In [ ]:
for tbi in tbi3s:
    print(wv3d(*(t*tbi)))

In [ ]:
wvS2 = euclid_to_sphere(wv3d)(r=1).trig_reduce()

In [ ]:
wvS2 = wvS2 / integrate(integrate(euclid_to_sphere(wv3d^2)(r=1)*sin(theta),(theta,0,pi)), (phi,0,2*pi))

In [ ]:
Pmag = density_plot(abs(wvS2)^2, (phi,-pi,pi), (theta,0,pi), cmap=cm.viridis,
                    plot_points=200,aspect_ratio=1,axes=False)

Pmag.show(axes_pad=0,aspect_ratio=1)

In [ ]:
P_curves = Graphics()

var('t')

# the 2-interations
for (wi, wj) in [(-tbi3s[0],-tbi3s[1]), (-tbi3s[0],-tbi3s[2]), (-tbi3s[0],tbi3s[3]), (-tbi3s[1],-tbi3s[2]), (-tbi3s[1],tbi3s[3]), (-tbi3s[2],tbi3s[3])]:
    three_curve = vector(cos(t)*wi+sin(t)*wj)
    three_curve = three_curve/three_curve.norm()
    #two_curve = vector(stereographic_proj(three_curve)).simplify()
    two_curve = vector(to_iso_sphere_coords(three_curve)).simplify()
    if (wi == -tbi3s[0] or wi == -tbi3s[1]) and wj == -tbi3s[2]:
        P_curves += parametric_plot(two_curve,
                             (t,0,arctan(1/2)-1e-2),
                             thickness=2,
                             color=rainbow(3)[0])
        P_curves += parametric_plot(two_curve,
                             (t,arctan(1/2)+1e-2,pi/2),
                             thickness=2,
                             color=rainbow(3)[0])
    else:
        P_curves += parametric_plot(two_curve,
                             (t,0,pi/2),
                             thickness=2,
                             color=rainbow(3)[0])
        
    if not( wi == -tbi3s[0] and wj == -tbi3s[1] ):
        P_curves += parametric_plot(two_curve,
                             (t,pi/2,pi),
                             thickness=2,
                             color=rainbow(3)[1])
    else:
        P_curves += parametric_plot(two_curve,
                             (t,pi/2,3*pi/4-1e-2),
                             thickness=2,
                             color=rainbow(3)[1])
        P_curves += parametric_plot(two_curve,
                             (t,3*pi/4+1e-2,pi),
                             thickness=2,
                             color=rainbow(3)[1])
        
    P_curves += parametric_plot(two_curve,
                         (t,pi,3*pi/2),
                         thickness=2,
                         color=rainbow(3)[2])
    if not( wi == -tbi3s[0] and wj == -tbi3s[1] ):
        P_curves += parametric_plot(two_curve,
                             (t,3*pi/2+1e-3,2*pi-1e-3),
                             thickness=2,
                             color=rainbow(3)[1])
    else:
        P_curves += parametric_plot(two_curve,
                             (t,3*pi/2, 7*pi/4-1e-4),
                             thickness=2,
                             color=rainbow(3)[1])
        P_curves += parametric_plot(two_curve,
                             (t,7*pi/4+1e-4,2*pi),
                             thickness=2,
                             color=rainbow(3)[1])
    

for wi in tbi3s:
    P_curves += point(to_iso_sphere_coords( wi), size=120, color='black')
    P_curves[-1].set_zorder(10)
    P_curves += point(to_iso_sphere_coords(-wi), size=120, color='black')
    P_curves[-1].set_zorder(10)

#P.show(aspect_ratio=1,axes=False, fig_tight=True)

In [ ]:
(P_curves+Pmag).show(axes_pad=0, aspect_ratio=1, figsize=12)